# 테스트 세트 데이터 전처리

In [1]:
import pandas as pd

test_df = pd.read_csv('./datas/test.csv') # 테스트용 데이터

In [4]:
test_df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

- 학습 데이터와 다르게 Fare에 null값이 있음 (평균값으로 대체)

In [ ]:
test_df['Fare'].fillna(test_df['Fare'].mean(), inplace=True)

In [5]:
# 알파벳으로 되어 있고, 각 호칭이 '.' 으로 끝나는 패턴 추출
test_df['Title'] = test_df.Name.str.extract('([A-Za-z]+)\.', expand = False)

In [8]:
test_df['Title'] = test_df['Title'].replace(['Mlle', 'Ms'], 'Miss')
test_df['Title'] = test_df['Title'].replace(['Mme'], 'Mrs')
test_df['Title'] = test_df['Title'].replace(['Dr', 'Rev', 'Major', 'Col', 'Countess', 'Capt', 'Sir', 'Lady', 'Don', 'Jonkheer', 'Dona'], 'Rare')
# 테스트 데이터에는 희귀 호칭으로 Dona가 있었음

In [9]:
test_df[['Title','Age']].groupby(['Title']).median()

,Age
Title,
Master,7.0
Miss,22.0
Mr,28.5
Mrs,36.5
Rare,44.0


In [11]:
test_df['Age'].fillna(test_df.groupby('Title')['Age'].transform('median'), inplace=True)

In [12]:
print(test_df['Age'].isnull().sum())

0


In [13]:
test_df.drop(['Cabin'], axis=1, inplace=True)

In [16]:
test_df['AgeBand'] = pd.cut(test_df['Age'], bins=8, labels=[1,2,3,4,5,6,7,8]) 

In [17]:
test_df['AgeBand']

0      4
1      5
2      7
3      3
4      3
      ..
413    3
414    5
415    5
416    3
417    1
Name: AgeBand, Length: 418, dtype: category
Categories (8, int64): [1 < 2 < 3 < 4 < 5 < 6 < 7 < 8]

In [18]:
test_df['Family'] = test_df['SibSp'] + test_df['Parch'] + 1 # 1은 나 자신을 추가해야 하기 때문

In [19]:
test_df['Alone'] = 0 # 기본값 혼자가 아님
test_df.loc[test_df['Family']==1, 'Alone'] = 1 # 부양가족수가 1이라면 1 반환

In [24]:
test_df['FareBand'] = pd.cut(test_df['Fare'], bins=5, labels=[1,2,3,4,5]) 

In [27]:
test_df.drop(['PassengerId', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare'], axis=1, inplace=True)

In [30]:
sex_mapping = {'male':1, 'female':2}
test_df['Sex'] = test_df['Sex'].map(sex_mapping).astype(int)

embarked_mapping = {'S':1, 'C':2, 'Q':3}
test_df['Embarked'] = test_df['Embarked'].map(embarked_mapping).astype(int)

new_mapping = {'Mr':1, 'Mrs':2, 'Miss':3, 'Master':4, 'Rare':5}
test_df['Title'] = test_df['Title'].map(new_mapping).astype(int)


In [31]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Pclass    418 non-null    int64   
 1   Sex       418 non-null    int32   
 2   Embarked  418 non-null    int32   
 3   Title     418 non-null    int32   
 4   AgeBand   418 non-null    category
 5   Family    418 non-null    int64   
 6   Alone     418 non-null    int64   
 7   FareBand  418 non-null    category
dtypes: category(2), int32(3), int64(3)
memory usage: 16.2 KB


In [32]:
# csv 파일로 저장
test_df.to_csv('./datas/test_df.csv')